In [195]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score,precision_recall_fscore_support
from Constants import *
from Preprocessing import *
from survival_losses import lognormal_loss,conditional_lognormal_loss
from Models import *
import copy
from Utils import *
#this is a subset of https://github.com/autonlab/auton-survival/tree/master. may remove dcm to reduce dependencies
from survival_models import dcm, dsm
import torch.nn as nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
data = DTDataset(use_smote=False)
data.processed_df[Const.timeseries_outcomes]

,OS (Calculated),Locoregional control (Time),FDM (months),time_to_event
id,,,,
3,6.033333,4.700000,6.033333,4.700000
5,7.333333,7.333333,7.333333,6.000000
6,7.466667,7.466667,7.466667,6.000000
7,7.800000,7.800000,7.800000,6.000000
8,8.066667,8.066667,8.066667,8.066667
...,...,...,...,...
10201,143.200000,143.200000,143.200000,143.200000
10202,144.366667,144.366667,144.366667,6.000000
10203,148.366667,148.366667,136.033333,136.033333


In [29]:
xtr,xtst,ytr,ytst = transition_sample(3,data)
xtr.shape

torch.Size([389, 83])

In [233]:
def create_representation(inputdim, layers, activation, bias=False):
    if activation == 'ReLU6': act = nn.ReLU6()
    elif activation == 'ReLU': act = nn.ReLU()
    elif activation == 'SeLU': act = nn.SELU()
    elif activation == 'Tanh': act = nn.Tanh()

    modules = []
    prevdim = inputdim

    for hidden in layers:
        modules.append(nn.Linear(prevdim, hidden, bias=bias))
        modules.append(act)
        prevdim = hidden
    return nn.Sequential(*modules)

class DSM(torch.nn.Module):

    def _init_dsm_layers(self, lastdim, n_outcomes):

        self.act = nn.Tanh()
        self.shape = torch.nn.ParameterList([nn.Parameter(torch.ones(self.k)) for i in range(n_outcomes)])
        self.scale = torch.nn.ParameterList([nn.Parameter(torch.ones(self.k)) for i in range(n_outcomes)])
     
        self.gate = torch.nn.ModuleList([nn.Sequential( nn.Linear(lastdim, self.k, bias=False) ) for i in range(n_outcomes)])

        self.scaleg =  torch.nn.ModuleList([nn.Sequential( nn.Linear(lastdim, self.k, bias=True) ) for i in range(n_outcomes)])

        self.shapeg = torch.nn.ModuleList([nn.Sequential( nn.Linear(lastdim, self.k, bias=False) ) for i in range(n_outcomes)])

    def __init__(self, inputdim, k=3, layers=None, dist='LogNormal',
               temp=1000., discount=1.0, optimizer='Adam',
               input_dropout=0,
               embedding_dropout=0.5,
                 activation='ReLU6',
                 n_outcomes=3,
                ):
        super().__init__()
        #I'm only using lognormal because weibull explosed
        self.k = k
        self.dist = dist
        self.temp = float(temp)
        self.discount = float(discount)
        self.optimizer = optimizer
        self.n_outcomes=n_outcomes

        if layers is None: layers = [1000]
        self.layers = layers

        if len(layers) == 0: lastdim = inputdim
        else: lastdim = layers[-1]

        self._init_dsm_layers(lastdim,n_outcomes)
        self.activation_name=activation
        self.embedding = create_representation(inputdim, layers, activation)
        self.input_dropout = nn.Dropout(input_dropout)
        self.embedding_dropout = nn.Dropout(embedding_dropout)
        self.squish = nn.LogSoftmax(dim=1)

    def _cdf(self,shape,scale,logits,t_horizon):
        k_ =  shape
        b_ = scale
        logits = self.squish(logits)
        if isinstance(t_horizon,float) or isinstance(t_horizon,int):
            t_horizon = [t_horizon]
        t_horz = torch.tensor(t_horizon).double().to(logits.device)
        t_horz = t_horz.repeat(shape.shape[0], 1)
        cdfs = []
        
        for j in range(len(t_horizon)):
            t = t_horz[:, j]
            lcdfs = []

            for g in range(self.k):
                mu = k_[:, g]
                sigma = b_[:, g]

                s = torch.div(torch.log(t) - mu, torch.exp(sigma)*np.sqrt(2))
                s = 0.5 - 0.5*torch.erf(s)
                s = torch.log(s)
                lcdfs.append(s)

            lcdfs = torch.stack(lcdfs, dim=1)
            lcdfs = lcdfs+logits
            lcdfs = torch.logsumexp(lcdfs, dim=1)
            cdfs.append(lcdfs)

        return cdfs

    def forward(self, x, t=None,**kwargs):
        """The forward function that is called when data is passed through DSM.

        Args:
          x:
            a torch.tensor of the input features.

        """
        x = self.input_dropout(x)
        xrep = self.embedding(x)
        xrep = self.embedding_dropout(xrep)
        dim = x.shape[0]
        
        shapes = []
        scales = []
        logitss = []
        survivals = []
        for i in range(self.n_outcomes):
            shape = self.act(self.shapeg[i](xrep)) + self.shape[i].expand(dim,-1)
            scale = self.act(self.scaleg[i](xrep)) + self.scale[i].expand(dim,-1)
            logits = self.gate[i](xrep)/self.temp
        
            if t is None:
                shapes.append(shape)
                scales.append(scale)
                logitss.append(logits)
            else:
                cdf = self._cdf(shape,scale,logits,t)
                survival = [torch.exp(c).T for c in cdf]
                survivals.append(survival)
        if t is None:
            return shapes, scales, logitss
        return survivals

    def get_shape_scale(self,i=None, **kwargs):
        if i is None:
            return(self.shape, self.scale)
        assert(i < self.n_outcomes and i >= 0)
        return (self.shape[i],self.scale[i])
    
test = DSM(xtr.shape[1])
test(xtr,42)[0], test.get_shape_scale(0)

([tensor([0.2411, 0.2563, 0.2586, 0.2398, 0.0926, 0.2780, 0.1258, 0.2425, 0.1349,
          0.2364, 0.2210, 0.0969, 0.1478, 0.1118, 0.2433, 0.1603, 0.1714, 0.0827,
          0.1958, 0.1780, 0.1214, 0.2280, 0.1447, 0.0909, 0.1439, 0.3231, 0.2110,
          0.0576, 0.1478, 0.2340, 0.2161, 0.2609, 0.2840, 0.2186, 0.1298, 0.2145,
          0.2073, 0.1926, 0.1290, 0.1581, 0.2466, 0.2025, 0.1468, 0.2398, 0.1728,
          0.1198, 0.1546, 0.0653, 0.1836, 0.2446, 0.2187, 0.2789, 0.2013, 0.2538,
          0.0162, 0.0536, 0.1278, 0.1473, 0.0899, 0.1406, 0.2324, 0.0946, 0.2540,
          0.1385, 0.1470, 0.1294, 0.0396, 0.0359, 0.2384, 0.2156, 0.1498, 0.0351,
          0.2155, 0.0481, 0.2357, 0.0812, 0.2084, 0.2777, 0.1958, 0.1724, 0.1889,
          0.2640, 0.1361, 0.1823, 0.0311, 0.0235, 0.0164, 0.1160, 0.2780, 0.2303,
          0.2138, 0.2067, 0.1564, 0.1295, 0.1384, 0.0952, 0.2723, 0.1478, 0.1117,
          0.1650, 0.0225, 0.1487, 0.0921, 0.1286, 0.1713, 0.2177, 0.2705, 0.2554,
          0.1881

In [ ]:
def format_tte_outcome(dataset,outcome,times=None, mintime = 1,maxtime = None,ids=None):
    #turns the timeseires censored output into an array of shape n_timepoints
    #time is by default 1 value until maxtime
    df = dataset.processed_df
    if ids is not None:
        df = df.loc[ids]
    values = df[outcome].values
    if maxtime is None:
        maxtime = values.max()
    event = values <= maxtime
    return torch.tensor(event), torch.tensor(values)

def format_tte_outcomes(dataset,outcomes,**kwargs):
    events = []
    times = []
    for outcome in outcomes:
        e,t = format_tte_outcome(dataset,outcome,**kwargs)
        events.append(e)
        times.append(t)
    events = torch.stack(events)
    times = torch.stack(times)
    print('events',events.shape)
    return events,times

def pretrain_dsm(dataset,
                 model=None,
                 outcomes = ['time_to_event'],
                 maxtime=72,lr=.01,
                 epochs=100000,
                 patience=10,
                 save_file=None,**model_kwargs):
    train_ids, test_ids = get_tt_split(dataset.processed_df.reset_index())
    
    state = 3
    xtrain = df_to_torch(dataset.get_input_state(step=state,ids=train_ids))
    #event series (1 = happend, 0 = didn't happen or already happened), t = 1-d list of times used of shape ytrain.shape[1]
    ytrain, ttrain = format_tte_outcomes(dataset,outcomes,ids=train_ids,maxtime=maxtime)
    if model is None:
        model = DSM(xtrain.shape[1],n_outcomes=len(outcomes),**model_kwargs)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_loss = 100000000000000000000000
    steps_since_improvement = 0
    if save_file is None: save_file= '../data/models/dsm_'+''.join(outcomes)+'.tar'
    for epoch in range(epochs):
        optimizer.zero_grad()
        curr_loss = 0
        for i in range(len(outcomes)):
            shape, scale = model.get_shape_scale(i)
            curr_loss += lognormal_loss(shape,scale,ttrain[i],ytrain[i],model.k)
        curr_loss.backward()
        print('pretrain epoch',epoch,'loss',curr_loss.item(),end='\r')
        optimizer.step()
        if curr_loss.item() < best_loss:
            best_loss = curr_loss.item()
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
            if steps_since_improvement > patience:
                break
    model.load_state_dict(torch.load(save_file))
    print('best pretrain loss',best_loss,'epochs',epoch)
    return model

from sklearn.metrics import roc_auc_score,f1_score,balanced_accuracy_score,matthews_corrcoef
def eval_model(model, xtest, ttest,ytest,timepoints = None,outcome_names=None):
    #ttest is time of event, ytest is if it happened at all
    if timepoints is None:
        timepoints = [12,24,36,48]
    ypreds = model(xtest,t=timepoints)
    allres = {}
    if outcome_names is None:
        outcome_names = [str(i) for i in range(model.n_outcomes)]
    for i, outcome in enumerate(outcome_names):
        res = {}
        for ii,time in enumerate(timepoints):
            ypred =ypreds[i][ii].cpu().detach().numpy()
            ytrue = (ttest[i] >= time).cpu().detach().numpy()
            entry = {}
            entry['roc_score'] = roc_auc_score(ytrue,ypred)
            entry['f1'] = f1_score(ytrue,ypred >= .5)
            entry['matthews'] = matthews_corrcoef(ytrue,ypred >= .5)
            res[time] = entry
        allres[outcome] = res
    return allres

def train_dsm(dataset, outcomes=Const.timeseries_outcomes,maxtime=72,save_file=None,main_epochs=1000,main_lr=.001,patience=10,**kwargs):
    #ok they do something different so like check this later?
    #the use this as a "premodel" and load the shape and scale weights?
    if save_file is None:
        save_file= '../data/models/dsm_'+''.join(outcomes)+'.tar'
        pretrain_save_file =  '../data/models/dsm_'+''.join(outcomes)+'_pretrain.tar'
        
    
    train_ids, test_ids = get_tt_split(dataset.processed_df.reset_index())
    
    state = 3
    xtrain = df_to_torch(dataset.get_input_state(step=state,ids=train_ids))
    xtest = df_to_torch(dataset.get_input_state(step=state,ids=test_ids))
    #n_outcomes by n_items, ytrain is event y/n, ttrain is time of event or last followup
    ytrain, ttrain = format_tte_outcomes(dataset,outcomes,ids=train_ids,maxtime=maxtime)
    ytest, ttest = format_tte_outcomes(dataset,outcomes,ids=test_ids,maxtime=maxtime)
    model = DSM(xtrain.shape[1],n_outcomes=len(outcomes),**kwargs)
    premodel = pretrain_dsm(dataset,outcomes=outcomes,maxtime=maxtime,save_file=pretrain_save_file,epochs=10,**kwargs)
    for i in range(premodel.n_outcomes):
        model.shape[i].data = premodel.shape[i].data
        model.scale[i].data = premodel.scale[i].data
    optimizer = torch.optim.Adam(model.parameters(),lr=main_lr)
    best_loss = 100000000000000
    best_metrics = {}
    steps_since_improvement = 0
    for epoch in range(main_epochs):
        optimizer.zero_grad()
        
        model.train()
        curr_loss = 0
        shapes,scales,logitss = model(xtrain)
        for i in range(model.n_outcomes):
            curr_loss += conditional_lognormal_loss(shapes[i],scales[i],logitss[i],ttrain[i],ytrain[i],model.k,discount=model.discount)/model.n_outcomes 
        curr_loss.backward()
        
        optimizer.step()
        with torch.no_grad():
            model.eval()
            shapes2,scales2,logitss2 = model(xtest)
            val_loss = 0
            for i in range(model.n_outcomes):
                val_loss += conditional_lognormal_loss(shapes2[i],scales2[i],logitss2[i],ttest[i],ytest[i],model.k,discount=model.discount)/model.n_outcomes 
            val_metrics= eval_model(model,xtest,ttest,ytest)
        print('val loss',val_loss.item())
        print('val metrics',val_metrics)
        print('____')
        if val_loss.item() < best_loss:
            best_loss = val_loss.item()
            best_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
            if steps_since_improvement > patience:
                break
    model.load_state_dict(torch.load(save_file))
    print('best_loss',best_loss)
    print(best_metrics)
    return model

from survival_losses import * 
test = train_dsm(data,k=10)
test(xtr,42)

,event,time
0,0,2029
1,1,4
2,1,47
3,1,133
4,0,2029
...,...,...
9100,0,350
9101,0,347
9102,0,346
9103,1,7
